In [22]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import pickle

### Data Loading

In [2]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df = df.copy()

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda d: d.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    cat_vars = ['PULocationID','DOLocationID']
    df[cat_vars] = df[cat_vars].astype(str)

    return df

In [4]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
num_vars = ['trip_distance']
cat_vars = ['PU_DO']#'PULocationID','DOLocationID']

### Training set
- DictVectorizer maps all the values into a sparse matrix (for string variables, it performs one-hot encoding)

In [13]:
dv = DictVectorizer()

train_dicts = df_train[cat_vars + num_vars].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_train[target].values

### Validation set

In [14]:
val_dicts = df_val[cat_vars + num_vars].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

### Model 1

In [15]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479582908966909

### Model 2

In [17]:
ridge = Ridge()
ridge.fit(X_train,y_train)

y_pred = ridge.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.342603943231458

### Save the best model

In [23]:
with open('models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,lr),f_out)